# Statistical Learning For Healthcare Data - Final Project - Diabetes

Group members: Giovanni Mele, Elisa Broserà

This preliminary notebook contains all the operations performed on the raw data to obtain a **cleaned** and **usable** version of the dataset, which serves as the **basis** for the project.

The main operations described below include:
- Handling of **invalid values**
- Removal of all **subsequent encounters** for each patient
- Handling of **missing values**
- **Simplification** of the **structure** of categorical variables
- Handling Diabetes treatments features

Notice that although in the notebook below we will try to respect this order, the way we proceeded was following the order, left to right, of the dataset columns. That's why `max_glu_serum` is not dropped in *Handling missing values/Exclusion of variables*, but later on in the code.

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

In [102]:
# dataset provided 
dataset = pd.read_csv("diabetic_data.csv")
# variable and category legend provided
legend = pd.read_csv("IDS_mapping.csv")

## Handling of Invalid Values

First of all, we notice that the dataset contains invalid values represented by the character `?`.  
To handle them more easily, we replace all `?` entries with `np.nan`. This allows for consistent treatment of missing data during preprocessing. 

In [103]:
dataset.replace("?", np.nan, inplace=True)

We also observe that the variable `race` has few occurences of the categories "Invalid" that we remove.

In [104]:
dataset= dataset[~(dataset.gender == 'Unknown/Invalid')]

## Removing Multiple Visits per Patient


We observe that some patients have multiple visit encounters recorded in the dataset. This behavior is also confirmed by the original papers related to the dataset.

We decide to keep **only the first encounter** for each patient and remove the others, to ensure **independent observations**.

In [105]:
dataset_sorted = dataset.sort_values(by='encounter_id')
dataset_first_visits = dataset_sorted.groupby('patient_nbr', as_index=False).nth(0)

In [106]:
# getting acquainted with the dataset
print("Dataset shape:", dataset_first_visits.shape)
print("Distribution of the target:", dataset_first_visits.readmitted.value_counts())
dataset_first_visits[dataset_first_visits.patient_nbr == 1660293]

Dataset shape: (71515, 50)
Distribution of the target: readmitted
NO     42982
>30    22240
<30     6293
Name: count, dtype: int64


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
269,2967810,1660293,Caucasian,Female,[60-70),NaN,6,25,7,10,...,No,Steady,No,No,No,No,No,No,Yes,>30


## Handling of Missing Values

We approach filling missing values using different strategies depending on the variable, among which:
- **Imputing** missing values in dataset_first_visits using **later visit** data
- **Exclusion of variables** with a low number of valid entries
- **Creating a missing category** containing NaN. 
The third strategy will be implemented below when simplifying categorical variables.

Let’s start with the variable `race`.

### Imputing missing values using later visit data

First, we observe that for the variable `race`, some missing values (`NaN`) in the dataset of first visits  
can be filled by using information from the original dataset. Specifically, if a patient has multiple visits,  
and the `race` is recorded in one or more later visits, we can use that information to fill the missing values in the first visit.

In [107]:
race_map = dataset.dropna(subset=['race']).drop_duplicates(subset='patient_nbr')[['patient_nbr', 'race']]
race_map = race_map.set_index('patient_nbr')['race']

dataset_first_visits['race'] = dataset_first_visits['race'].fillna(dataset_first_visits['patient_nbr'].map(race_map))

<ipython-input-107-d6130aefa949>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_first_visits['race'] = dataset_first_visits['race'].fillna(dataset_first_visits['patient_nbr'].map(race_map))


Since the issue of missing values in `race` persists after attempts to fill them, we will later drop the rows where `race` remains missing.  
Filling these missing values with the most frequent category would likely introduce bias.

In [108]:
print("Number of NaN for race:", dataset_first_visits.race.isna().sum())

Number of NaN for race: 1848


### Exclusion of variables

We go on quantifying the missing data across the dataset.

Note that here we do **not** consider missing data from **categorical variables** such as `admission_type_id`. We handle these separately later.

In [109]:
# quantifying NaN 
nan_percentages = dataset_first_visits.isna().mean() * 100
print(nan_percentages)

encounter_id                 0.000000
patient_nbr                  0.000000
race                         2.584073
gender                       0.000000
age                          0.000000
weight                      96.010627
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                  43.406278
medical_specialty           48.206670
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.015381
diag_2                       0.411103
diag_3                       1.712927
number_diagnoses             0.000000
max_glu_serum               95.167447
A1Cresult                   81.841572
metformin                    0.000000
repaglinide                  0.000000
nateglinide 

In [110]:
# variables with more than 20% NaN
print(nan_percentages > 20)

encounter_id                False
patient_nbr                 False
race                        False
gender                      False
age                         False
weight                       True
admission_type_id           False
discharge_disposition_id    False
admission_source_id         False
time_in_hospital            False
payer_code                   True
medical_specialty            True
num_lab_procedures          False
num_procedures              False
num_medications             False
number_outpatient           False
number_emergency            False
number_inpatient            False
diag_1                      False
diag_2                      False
diag_3                      False
number_diagnoses            False
max_glu_serum                True
A1Cresult                    True
metformin                   False
repaglinide                 False
nateglinide                 False
chlorpropamide              False
glimepiride                 False
acetohexamide 

From the group of variables with a consistent amount of missing data (threshold of 20%),  we decide to remove the variables `weight` and `payer_id`, following the recommendations of previous papers and analyses using this dataset.

Other variables are initially kept due to their potential significance in the analysis. `max_glu_serum`, `medical_specialty`  and `A1Cresult` also have missing values exceeding the threshold, but they will be removed later in the code.

In [111]:
dataset_first_visits.drop(['weight', 'payer_code'], axis=1, inplace=True)


<ipython-input-111-2c40c8371770>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_first_visits.drop(['weight', 'payer_code'], axis=1, inplace=True)


## Simplification of the Structure of Categorical Variables

### Dealing with `admission_type_id`, `admission_source_id`, `discharge_disposition_id`

We deal with the categories  `admission_type_id`, `admission_source_id`, and `discharge_disposition_id` by creating an "other" category for grouping, as done in *Beata Strack et al. (2014)*.

However, we slightly differentiate our approach:  while their method includes "Not Mapped" and all categories except the two most frequent into "other,"  
we first remove "NULL", "Unknown" and very infrequent categories, then we aggregate the remaining ones into "other."

The idea behind this is to avoid grouping too heterogeneous information in the "other" category.

In [112]:
# function to visualize for each categorical feature a contingency table, including also the total number of observations for that variable
def summary_categorical_variable(df, var_categorica, target="readmitted"):
    frequenze = pd.crosstab(df[var_categorica], df[target])

    frequenze['Totale'] = frequenze.sum(axis=1)

    colonne_ordinate = ['Totale'] + [col for col in frequenze.columns if col != 'Totale']
    frequenze = frequenze[colonne_ordinate]

    return frequenze


We show the contingency table for `admission_type_id`:

In [113]:
summary_admission_type_id = summary_categorical_variable(dataset_first_visits, "admission_type_id")
summary_admission_type_id

readmitted,Totale,<30,>30,NO
admission_type_id,,,,
1,36488,3262,11450,21776
2,13028,1149,4073,7806
3,13916,1143,3732,9041
4,9,1,2,6
5,3174,265,1039,1870
6,4588,452,1875,2261
7,21,0,0,21
8,291,21,69,201


We define as infrequent categories the ones with less than 200 patients.

In [114]:
def infrequent_categories(summary, threshold = 200, N=71518):
    infrequent_list = [] 
    for subcategory,tot in zip(summary.index, summary["Totale"]): 
        if tot < threshold:
            print(f"subcategory {subcategory} is infrequent")
            infrequent_list.append(subcategory)
    return infrequent_list

In [115]:
print("Infrequent categories for admission_type_id:")
infrequent_categories(summary_admission_type_id)

Infrequent categories for admission_type_id:
subcategory 4 is infrequent
subcategory 7 is infrequent


[4, 7]

For the sake of completeness, we visualize all infrequent categories for all categorical variables, but we will remove them only for these 3 variables.

In [116]:
categorical_variables = dataset_first_visits.select_dtypes(include=['object', 'category']).columns.tolist()

In [117]:
# Categorical variables with numerical (non-string) category labels
categorical_variables.append('admission_type_id')
categorical_variables.append('discharge_disposition_id')
categorical_variables.append('admission_source_id')

In [118]:
infrequent_subcategories_for_cat_var = {}

In [119]:
# infrequent categories for each categorical variable
for variable in categorical_variables: 
    print(f"\n Analyzing variable {variable}")
    summary_variable = summary_categorical_variable(dataset_first_visits, variable, target="readmitted")
    inf = infrequent_categories(summary_variable)
    if len(inf) >= 1:
        infrequent_subcategories_for_cat_var[variable] = inf


 Analyzing variable race

 Analyzing variable gender

 Analyzing variable age
subcategory [0-10) is infrequent

 Analyzing variable medical_specialty
subcategory AllergyandImmunology is infrequent
subcategory Anesthesiology is infrequent
subcategory Anesthesiology-Pediatric is infrequent
subcategory Cardiology-Pediatric is infrequent
subcategory DCPTEAM is infrequent
subcategory Dentistry is infrequent
subcategory Dermatology is infrequent
subcategory Endocrinology is infrequent
subcategory Endocrinology-Metabolism is infrequent
subcategory Gynecology is infrequent
subcategory Hematology is infrequent
subcategory Hematology/Oncology is infrequent
subcategory Hospitalist is infrequent
subcategory InfectiousDiseases is infrequent
subcategory Neurology is infrequent
subcategory Neurophysiology is infrequent
subcategory Obsterics&Gynecology-GynecologicOnco is infrequent
subcategory Obstetrics is infrequent
subcategory Ophthalmology is infrequent
subcategory Osteopath is infrequent
subcate

In [120]:
print("Infrequent categories for admission_type_id:")
infrequent_subcategories_for_cat_var['admission_type_id']

Infrequent categories for admission_type_id:


[4, 7]

### Removing "NULL", "Unknown", infrequent categories

Here below, we proceed first by removing "NULL" and "Unknown".
For these variables, we consider "NULL" and "Unknown" as missing values:  
- "NULL" because it represents an invalid value,  
- "Unknown" because this classification is reasonable, also based on discussions with the tutors.

In [121]:
# counting invalid values
print("admission_type_id:", dataset_first_visits[(dataset_first_visits.admission_type_id == 5) | (dataset_first_visits.admission_type_id == 6)].shape[0])
print("discharge_disposition_id:", dataset_first_visits[(dataset_first_visits.discharge_disposition_id == 18) | (dataset_first_visits.discharge_disposition_id == 26)].shape[0])
print("admission_source_id:", dataset_first_visits[(dataset_first_visits.admission_source_id == 17) | (dataset_first_visits.admission_source_id == 21)].shape[0])

admission_type_id: 7762
discharge_disposition_id: 2474
admission_source_id: 4949


In [122]:
# removing NULL and Unknown
dataset_without_na = dataset_first_visits[~((dataset_first_visits.admission_type_id == 5) |
                                             (dataset_first_visits.admission_type_id == 6) |
                                               (dataset_first_visits.discharge_disposition_id == 18) |
                                                 (dataset_first_visits.discharge_disposition_id == 26) |
                                                   (dataset_first_visits.admission_source_id == 17) |
                                                     (dataset_first_visits.admission_source_id == 21))]
print("After removing invalid values:",dataset_without_na.shape)

After removing invalid values: (60724, 48)


Then we remove infrequent categories.

In [123]:
# removing infrequent categories
dataset_without_infrequent_categories = dataset_without_na[~( (dataset_without_na['admission_type_id'].isin(infrequent_subcategories_for_cat_var['admission_type_id'])) |
                                      (dataset_without_na['discharge_disposition_id'].isin(infrequent_subcategories_for_cat_var['discharge_disposition_id'])) |
                                        (dataset_without_na['admission_source_id'].isin(infrequent_subcategories_for_cat_var['admission_source_id'])))]
print("After removing also infrequent categories:", dataset_without_infrequent_categories.shape)

After removing also infrequent categories: (60139, 48)


### Gathering categories in `admission_type_id`, `admission_source_id`, `discharge_disposition_id`

After removing the NULL/Unknown/infrequent categories from these three variables, the next step is to **simplify their structure**, following the approach proposed by *Beata Strack et al. (2014)*.

The main goal is to **minimize the number of categories** for each variable as much as possible, since a large number of categories can increase model complexity—especially when using techniques like `OneHotEncoder`.

In [124]:
#creation of a copy of the dataset up to now 
dataset_without_infrequent_categories_2 = dataset_without_infrequent_categories.copy()

Let's start with `discharge_disposition_id`.

First of all, patients classified as expired in the `discharge_disposition_id` variable, as well as those discharged to a hospice, are considered medical outliers and therefore need to be excluded from the analysis. As a matter of fact, expired patients will not be readmitted and hospice ones extremely unlikely will.

In [125]:
hospice = [13, 14]
dataset_without_infrequent_categories_2['discharge_disposition_id'] = dataset_without_infrequent_categories['discharge_disposition_id'].replace(dict.fromkeys(hospice, 'hospice'))

In [126]:
# removing expired patients
dataset_without_infrequent_categories_2 = dataset_without_infrequent_categories_2[~(dataset_without_infrequent_categories_2.discharge_disposition_id == 11)] 

In [127]:
# removing hospice patients
dataset_without_infrequent_categories_2 = dataset_without_infrequent_categories_2[~(dataset_without_infrequent_categories_2.discharge_disposition_id == 'hospice')] 

Now, let's move finally to the recategorization part.

In [128]:
another_medical_facilities = [2,3,4,5,22,23]
dataset_without_infrequent_categories_2['discharge_disposition_id'] = dataset_without_infrequent_categories_2['discharge_disposition_id'].replace(dict.fromkeys(another_medical_facilities, 'another_medical_facility'))

In [129]:
dataset_without_infrequent_categories_2.discharge_disposition_id.value_counts()

discharge_disposition_id
1                           39804
another_medical_facility    11535
6                            7069
7                             366
25                             17
Name: count, dtype: int64

In [130]:
to_home = [1,6]
dataset_without_infrequent_categories_2['discharge_disposition_id'] = dataset_without_infrequent_categories_2['discharge_disposition_id'].replace(dict.fromkeys(to_home, 'to_home'))

In [131]:
dataset_without_infrequent_categories_2.discharge_disposition_id.value_counts()

discharge_disposition_id
to_home                     46873
another_medical_facility    11535
7                             366
25                             17
Name: count, dtype: int64

In [132]:
other = [7,25]
dataset_without_infrequent_categories_2['discharge_disposition_id'] = dataset_without_infrequent_categories_2['discharge_disposition_id'].replace(dict.fromkeys(other, 'other'))

Let's see for clarification the final structure of `discharge_disposition_id`.

In [133]:
dataset_without_infrequent_categories_2.discharge_disposition_id.value_counts()

discharge_disposition_id
to_home                     46873
another_medical_facility    11535
other                         383
Name: count, dtype: int64

Now let's move to `admission_source_id`:

In [134]:
dataset_without_infrequent_categories_2.admission_source_id.value_counts()


admission_source_id
7    34454
1    19204
4     2126
6     1763
2      808
5      436
Name: count, dtype: int64

In [135]:
another_medical_facility_other = [4,5,6]
dataset_without_infrequent_categories_2['admission_source_id'] = dataset_without_infrequent_categories_2['admission_source_id'].replace(dict.fromkeys(another_medical_facility_other, 'another_medical_facility/other'))

In [136]:
referral = [1,2]
dataset_without_infrequent_categories_2['admission_source_id'] = dataset_without_infrequent_categories_2['admission_source_id'].replace(dict.fromkeys(referral, 'referral'))

In [137]:
emergency = [7]
dataset_without_infrequent_categories_2['admission_source_id'] = dataset_without_infrequent_categories_2['admission_source_id'].replace(dict.fromkeys(emergency, 'emergency'))

The final structure for the variable is:

In [138]:
dataset_without_infrequent_categories_2.admission_source_id.value_counts()


admission_source_id
emergency                         34454
referral                          20012
another_medical_facility/other     4325
Name: count, dtype: int64

As for `admission_type_id`, we do not recategorize it, as there are only 4 categories.
We only align `admission_type_id` with the format of the other variables. 

In [139]:
emergency = [1]
urgent = [2]
elective = [3]
other = [8]
dataset_without_infrequent_categories_2['admission_type_id'] = dataset_without_infrequent_categories_2['admission_type_id'].replace(dict.fromkeys(emergency, 'emergency'))
dataset_without_infrequent_categories_2['admission_type_id'] = dataset_without_infrequent_categories_2['admission_type_id'].replace(dict.fromkeys(urgent, 'urgent'))
dataset_without_infrequent_categories_2['admission_type_id'] = dataset_without_infrequent_categories_2['admission_type_id'].replace(dict.fromkeys(elective, 'elective'))
dataset_without_infrequent_categories_2['admission_type_id'] = dataset_without_infrequent_categories_2['admission_type_id'].replace(dict.fromkeys(other, 'other'))

The final structure for `admission_type_id` is:

In [140]:
dataset_without_infrequent_categories_2.admission_type_id.value_counts()


admission_type_id
emergency    33557
elective     12919
urgent       12026
other          289
Name: count, dtype: int64

Now let's move back to the variable `race`, removing the observations associated to the remaining missing values.

In [141]:
print("Remained NaN:")
dataset_without_infrequent_categories_2.isna().sum()

Remained NaN:


encounter_id                    0
patient_nbr                     0
race                         1639
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
medical_specialty           27961
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          9
diag_2                        268
diag_3                       1008
number_diagnoses                0
max_glu_serum               58663
A1Cresult                   47764
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [142]:
dataset_without_infrequent_categories_2 = dataset_without_infrequent_categories_2.dropna(subset=['race'])

### Handling missing values for influential variables (`A1cResult` and `medical_specialty`)

Based on the work of *Beata Strack et al.* (2014), we decide to handle missing values for the HbA1c test and for medical specialty differently than before, as these variables are considered strong predictors.

Specifically, we do not drop any missing values; instead, we replace them with a dedicated "missing" category.

The high number of missing values of `A1cResult` may be due to the fact that in certain cases of hospitalization for critical conditions, the test was not conducted because it was not considered a priority.

Another reasonable hypothesis is that some missing data are associated to patients for which there is such an high evidence of diabetes that the test is unnecessary.

In [143]:
dataset_without_infrequent_categories_2['medical_specialty'] = dataset_without_infrequent_categories_2['medical_specialty'].fillna('missing')

In [144]:
dataset_without_infrequent_categories_2['A1Cresult'] = dataset_without_infrequent_categories_2['A1Cresult'].fillna('missing')

In [145]:
dataset_without_infrequent_categories_2.A1Cresult.value_counts()

A1Cresult
missing    46463
>8          4999
Norm        3296
>7          2394
Name: count, dtype: int64

In [146]:
dataset_without_infrequent_categories_2.medical_specialty.value_counts()

medical_specialty
missing                   27099
InternalMedicine           9050
Emergency/Trauma           4344
Family/GeneralPractice     3528
Cardiology                 3205
                          ...  
SportsMedicine                1
Proctology                    1
Perinatology                  1
Neurophysiology               1
Resident                      1
Name: count, Length: 69, dtype: int64

For `max_glu_serum`, we decide to drop the variable as it has more than $90 \%$ of N.A. 

In [147]:
dataset_without_infrequent_categories_3 = dataset_without_infrequent_categories_2.drop('max_glu_serum', axis=1)

### Dealing with missing values of `diag_1`, `diag_2`, `diag_3`

Variable `diag_1` presents at this point a small amount of N.A., so we drop them.

Moreover, we decide to drop `diag_2` and `diag_3` and to keep only the information deriving from the principal diagnosis, as we think that it's the most significant among the 3 and the other 2 could be misleading for predicting the target.

In [148]:
dataset_without_infrequent_categories_3[dataset_without_infrequent_categories_3['diag_1'].isna()]

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1006,7599132,9946782,Caucasian,Male,[80-90),elective,another_medical_facility,another_medical_facility/other,3,Urology,...,No,No,No,No,No,No,No,No,No,>30
1267,8927178,520452,Caucasian,Male,[60-70),elective,to_home,referral,3,Urology,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
1488,10122996,3650130,AfricanAmerican,Male,[60-70),elective,to_home,referral,5,Urology,...,No,No,No,No,No,No,No,Ch,Yes,NO
3197,20095914,916947,AfricanAmerican,Male,[70-80),elective,another_medical_facility,referral,1,Urology,...,No,Steady,No,No,No,No,No,No,Yes,>30
37693,117010956,25300467,Hispanic,Male,[40-50),emergency,another_medical_facility,referral,4,missing,...,No,Steady,No,No,No,No,No,Ch,Yes,<30
57058,163172034,27758448,Caucasian,Female,[80-90),emergency,to_home,referral,2,missing,...,No,Steady,No,No,No,No,No,No,Yes,NO
57737,164326842,39688524,AfricanAmerican,Male,[40-50),elective,other,emergency,3,Emergency/Trauma,...,No,Down,No,No,No,No,No,Ch,Yes,NO
60314,169067490,59785542,AfricanAmerican,Female,[80-90),emergency,to_home,emergency,8,missing,...,No,Up,No,No,No,No,No,Ch,Yes,NO
86018,273014598,113173146,Hispanic,Male,[60-70),emergency,another_medical_facility,referral,4,InternalMedicine,...,No,No,No,No,No,No,No,No,Yes,<30


In [149]:
dataset_without_infrequent_categories_3.dropna(subset=['diag_1'], inplace=True)

In [150]:
dataset_without_infrequent_categories_4 = dataset_without_infrequent_categories_3.drop(['diag_2','diag_3'], axis=1)

At this point, handling of missing values is concluded.

In [151]:
dataset_without_infrequent_categories_4.isna().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
number_diagnoses            0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide

In [152]:
dataset_without_infrequent_categories_4.shape

(57143, 45)

### Recategorization of `medical_specialy` and `diag_1`

This part is done following *Beata Strack et al.* (2014), where medical specialties (or principal diagnoses) are grouped in classes.

The main goal is to **simplify the structure** of these variables as much as possible, in a logical and coherent way.

In [153]:
dataset_without_infrequent_categories_4['medical_specialty'] = dataset_without_infrequent_categories_4['medical_specialty'].apply(
    lambda x: 'Surgery' if x.startswith('Sur') else x
)

In [154]:
dataset_without_infrequent_categories_4.medical_specialty.value_counts()

medical_specialty
missing                                 27096
InternalMedicine                         9049
Emergency/Trauma                         4343
Family/GeneralPractice                   3528
Cardiology                               3205
Surgery                                  3199
Orthopedics                               896
Radiologist                               800
Nephrology                                667
Orthopedics-Reconstructive                632
ObstetricsandGynecology                   495
Psychiatry                                475
Pulmonology                               473
Urology                                   413
Gastroenterology                          332
Oncology                                  202
Neurology                                 162
Pediatrics                                155
PhysicalMedicineandRehabilitation         146
Pediatrics-Endocrinology                  133
Endocrinology                              86
Otolaryngology  

In [155]:
kept_categories = ['missing', 'Surgery', 'Cardiology', 'Emergency/Trauma','Family/GeneralPractice','InternalMedicine' ]

dataset_without_infrequent_categories_4['medical_specialty'] = dataset_without_infrequent_categories_4['medical_specialty'].apply(
    lambda x: x if x in kept_categories else 'other'
)


Let's see the final structure of `medical_specialy`

In [156]:
dataset_without_infrequent_categories_4.medical_specialty.value_counts()

medical_specialty
missing                   27096
InternalMedicine           9049
other                      6723
Emergency/Trauma           4343
Family/GeneralPractice     3528
Cardiology                 3205
Surgery                    3199
Name: count, dtype: int64

Now for `diag_1`...

In [157]:
circulatory = [i for i in range(390,460)]
circulatory.append(785)
circulatory_str = [str(num) for num in circulatory]

In [158]:
respiratory = [i for i in range(460,520)]
respiratory.append(786)
respiratory_str = [str(num) for num in respiratory]

In [159]:
digestive = [i for i in range(520,580)]
digestive.append(787)
digestive_str = [str(num) for num in digestive]

In [160]:
injury = [i for i in range(800,1000)]
injury_str = [str(num) for num in injury]

In [161]:
muscoloskeletal = [i for i in range(710,740)]
muscoloskeletal_str = [str(num) for num in muscoloskeletal]

In [162]:
genitourinary = [i for i in range(580,630)]
genitourinary.append(788)
genitourinary_str = [str(num) for num in genitourinary]

In [163]:
neoplasms = [i for i in range(140,240)]

neoplasms_str = [str(num) for num in neoplasms]

In [164]:
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].apply(
    lambda x: 'diabetes' if x.startswith('250') else x
)

In [165]:
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(circulatory_str, 'circulatory'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(respiratory_str, 'respiratory'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(digestive_str, 'digestive'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(injury_str, 'injury'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(muscoloskeletal_str, 'muscoloskeletal'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(genitourinary_str, 'genitourinary'))
dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].replace(dict.fromkeys(neoplasms_str, 'neoplasms'))

In [166]:
kept_categories_diag_1 = ['circulatory', 'respiratory', 'digestive', 'injury','muscoloskeletal','genitourinary', 'neoplasms', 'diabetes' ]

dataset_without_infrequent_categories_4['diag_1'] = dataset_without_infrequent_categories_4['diag_1'].apply(
    lambda x: x if x in kept_categories_diag_1 else 'other'
)


Let's see the final structure of `diag_1`:

In [167]:
dataset_without_infrequent_categories_4.diag_1.value_counts()

diag_1
circulatory        17447
other               9853
respiratory         7640
digestive           5308
diabetes            4761
injury              3881
muscoloskeletal     3233
genitourinary       2898
neoplasms           2122
Name: count, dtype: int64

## Handling of diabetes treatments features

The second half of the dataset includes a collection of potential **diabetes treatments**, indicating whether each treatment is prescribed to the patient and, if so, whether the dosage is kept steady, increased, or reduced. Moreover, an additional binary variable indicates if at least one of the treatments is prescribed to the patient.

Our idea is to condense all of these information in 3 numerical variables: 
- `n_treatments` indicating the total number of treatments for each patient
- `rate_up_treatments` indicating the number of time a dosage has been increased
- `rate_down_treatments` indicating the number of time a dosage has been decreased

In [168]:
dataset_without_infrequent_categories_4.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'number_diagnoses',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Let's collect all the variables related to the treatments into a new list.

In [169]:
drugs_col = dataset_without_infrequent_categories_4.columns[19:19+23]

In [170]:
drugs_col

Index(['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone'],
      dtype='object')

Before creating the synthetic variables, we analyze each treatment by computing the **entropy** of them.

The idea is to see if there is any treatment worth keeping in the dataset.

In [171]:
H = []
drugs_col = dataset_without_infrequent_categories_4.columns[19:19+23]
for col in drugs_col:
    freq = dataset_without_infrequent_categories_4[col].value_counts(normalize=True)
    h = entropy(freq)
    print(f"Entropy of {col}: {h}")
    H.append(h)


Entropy of metformin: 0.5927228932911752
Entropy of repaglinide: 0.08283815725117809
Entropy of nateglinide: 0.04914354754378707
Entropy of chlorpropamide: 0.00592255996606079
Entropy of glimepiride: 0.23838066115459558
Entropy of acetohexamide: 0.0002091822870168281
Entropy of glipizide: 0.4363011689722041
Entropy of glyburide: 0.3827350921927864
Entropy of tolbutamide: 0.0018393946263106883
Entropy of pioglitazone: 0.28967449817031116
Entropy of rosiglitazone: 0.25849413155126766
Entropy of acarbose: 0.01882078239386435
Entropy of miglitol: 0.0029889114036732852
Entropy of troglitazone: 0.0003941041771104566
Entropy of tolazamide: 0.0024268407455257505
Entropy of examide: 0.0
Entropy of citoglipton: 0.0
Entropy of insulin: 1.1871066359486862
Entropy of glyburide-metformin: 0.04863913695230127
Entropy of glipizide-metformin: 0.0012258961802740288
Entropy of glimepiride-pioglitazone: 0.0
Entropy of metformin-rosiglitazone: 0.0
Entropy of metformin-pioglitazone: 0.0002091822870168281


The entropy in this case of 4 categories for each treatment goes from 0 to $\log_2 (4) = 2$. **Insulin** is the only with a consistent value of entropy ($>1$), so we decide to keep it.

Now we create the **3 synthetic variables** that will replace all the others apart from `insulin`.

In [172]:
dataset_without_infrequent_categories_4['n_up'] = (dataset_without_infrequent_categories_4[drugs_col] == 'Up').sum(axis=1)
dataset_without_infrequent_categories_4['n_steady'] = (dataset_without_infrequent_categories_4[drugs_col] == 'Steady').sum(axis=1)
dataset_without_infrequent_categories_4['n_down'] = (dataset_without_infrequent_categories_4[drugs_col] == 'Down').sum(axis=1)
dataset_without_infrequent_categories_4['n_no'] = (dataset_without_infrequent_categories_4[drugs_col] == 'No').sum(axis=1)


In [173]:
dataset_without_infrequent_categories_4['n_treatments'] = dataset_without_infrequent_categories_4['n_up'] + dataset_without_infrequent_categories_4['n_steady'] + dataset_without_infrequent_categories_4['n_down']

In [174]:
dataset_without_infrequent_categories_4['rate_down_treatments'] = dataset_without_infrequent_categories_4['n_down'] / dataset_without_infrequent_categories_4['n_treatments']

In [175]:
dataset_without_infrequent_categories_4['rate_up_treatments'] = dataset_without_infrequent_categories_4['n_up'] / dataset_without_infrequent_categories_4['n_treatments']

For the patient without any active treatment, we set the two rates to 0.

In [176]:
dataset_without_infrequent_categories_4[['rate_down_treatments','rate_up_treatments']] = dataset_without_infrequent_categories_4[['rate_down_treatments','rate_up_treatments']].fillna(0)

In [177]:
dataset_without_infrequent_categories_4.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'number_diagnoses',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted', 'n_up',
       'n_steady', 'n_down', 'n_no', 'n_treatments', 'rate_down_treatments',
       'rate_up_treatments'],
      dtype='object')

Now let's remove all the variables related to medicines that are no longer needed at this stage.

In [178]:
drugs_col_tot = drugs_col
drugs_col = list(drugs_col)
drugs_col.remove('insulin')

In [179]:
dataset_without_infrequent_categories_5 = dataset_without_infrequent_categories_4.drop(drugs_col, axis = 1)

In [180]:
dataset_without_infrequent_categories_6 = dataset_without_infrequent_categories_5.drop(['patient_nbr','encounter_id','change','diabetesMed'], axis = 1)

Let's remove the auxiliary variables used.

In [181]:
dataset_without_infrequent_categories_7 = dataset_without_infrequent_categories_6.drop(['n_no','n_down','n_steady','n_up'], axis = 1)

In [182]:
dataset_without_infrequent_categories_7.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'number_diagnoses', 'A1Cresult',
       'insulin', 'readmitted', 'n_treatments', 'rate_down_treatments',
       'rate_up_treatments'],
      dtype='object')

In [183]:
print("Final shape of the dataset:")
dataset_without_infrequent_categories_7.shape

Final shape of the dataset:


(57143, 22)

In [184]:
# saving the final dataset
dataset_without_infrequent_categories_7.to_csv("final_dataset_1",index=False )

In [185]:
df = pd.read_csv("final_dataset_1")

In [186]:
df

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,number_emergency,number_inpatient,diag_1,number_diagnoses,A1Cresult,insulin,readmitted,n_treatments,rate_down_treatments,rate_up_treatments
0,Caucasian,Female,[80-90),urgent,to_home,another_medical_facility/other,13,missing,68,2,...,0,0,circulatory,8,missing,Steady,NO,2,0.0,0.0
1,Caucasian,Female,[90-100),elective,another_medical_facility,another_medical_facility/other,12,InternalMedicine,33,3,...,0,0,circulatory,8,missing,Steady,NO,2,0.0,0.0
2,Caucasian,Male,[40-50),emergency,to_home,emergency,1,missing,51,0,...,0,0,neoplasms,5,missing,Steady,NO,2,0.0,0.0
3,AfricanAmerican,Female,[40-50),emergency,to_home,emergency,9,missing,47,2,...,0,0,diabetes,9,missing,Steady,>30,1,0.0,0.0
4,Caucasian,Male,[50-60),urgent,to_home,referral,3,missing,31,6,...,0,0,circulatory,9,missing,Steady,>30,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57138,Caucasian,Female,[70-80),emergency,to_home,emergency,9,missing,50,2,...,0,0,digestive,9,>7,Steady,>30,2,0.0,0.5
57139,Other,Female,[40-50),emergency,to_home,emergency,14,missing,73,6,...,1,0,genitourinary,9,>8,Up,>30,2,0.0,0.5
57140,Other,Female,[60-70),emergency,to_home,emergency,2,missing,46,6,...,1,1,injury,9,missing,Steady,>30,1,0.0,0.0
57141,Caucasian,Female,[80-90),emergency,to_home,emergency,5,missing,76,1,...,1,0,other,9,missing,Up,NO,1,0.0,1.0
